# Hands-on Day 1

- course [slides](https://github.com/leggerf/MLCourse-2122/blob/master/Slides/Day1/Big%20data%20science%20-%20Day%201%20-%202122.pdf)

# You'll learn

- familiarize with jupyter notebooks, numpy and pandas

## Input data
- efficient data format: convert CSV to Parquet
- create input vector with features for MLLib. Format of the input depends on chosen ML library

## Visualization
- explore dataset, plot features
- correlation matrix


# Dataset description

The dataset used in this example is described [here](https://archive.ics.uci.edu/ml/datasets/HIGGS). It is a binary classification problem where the goal is to train a classifier able to distinguish between a signal process, the production of new theoretical Higgs bosons, and a background process with identical decay products but distinct kinematic features.

Each row of this dataset contains 28 features plus the label:

- 21 low-level features which represent the basic measure made by the particle detector
  - Momentum of the observed paricles
  - Missing transverse momentum
  - Jets and b-tagging information
- 7 high-level features computed from the low-level features that encode the knowledge of the different intermediate states of the two processes (reconstructed invariant masses)

## Prepare the execution environment

Your code will run on a single dedicated server with 24 cores (hyperthreading enabled) and 192 GB of RAM. 
All the services needed for this tutorial are deployed as Kubernetes applications on this server. These include:
* JupytherHub
* Jupyter single-user servers
* the HDFS file-system
* Spark Clusters on demand 

#### Load custom magics definition

We load an external file implemanting some custom *magics* function. Have a look at it.

In [1]:
%reload_ext custom_magics

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Check out these custom functions
from custom_functions import *

#### Load the Spark context

We use the custom magic *%sc* to load a pre-defined Spark context.

In [3]:
num_workers=5
spark=%sc 5

from pyspark.sql import SparkSession
spark_session = SparkSession(spark)

#check if spark is there
spark

<SparkContext master=k8s://https://192.168.2.39:6443 appName=jupyter-leggerf>

In [4]:
# Check number of workers (executors), and cores per executor

executor_count = len(spark._jsc.sc().statusTracker().getExecutorInfos()) - 1
cores_per_executor = int(spark.getConf().get('spark.executor.cores','1'))

print('Number of executors: '+ str(executor_count))
print('Cores per executor: '+ str(cores_per_executor))

Number of executors: 5
Cores per executor: 5


# Exercise 1

## Get familiar with kubernetes commands

- You can open a terminal and use the commands: 
  - `kubectl get pods`
  - `kubectl describe pod PODNAME`
  - `kubectl get nodes`
  - `kubectl describe node NODENAME`
  - `kubectl describe farm`

- Try to stop spark with `spark.stop()`, and start it again running the cell above with a different number of workers. What happens? You can play with the number of workers, and run the cells below that execute spark commands. Provided you're getting all the workers you're asking for, does the execution time change? Try to make some scaling tests

- if you don't stop spark correctly, you will see pods in Error state. You can get rid of those by running this command:
    `kubectl get pods -n YOURUSERNAME | grep Error | awk '{print $1}' | xargs kubectl delete pod -n YOURUSERNAME`

In [5]:
!kubectl get pods

NAME                                      READY   STATUS    RESTARTS   AGE
jupyter-leggerf-584f887f6dffe160-exec-1   1/1     Running   0          7s
jupyter-leggerf-584f887f6dffe160-exec-2   1/1     Running   0          7s
jupyter-leggerf-584f887f6dffe160-exec-3   1/1     Running   0          7s
jupyter-leggerf-584f887f6dffe160-exec-4   1/1     Running   0          7s
jupyter-leggerf-584f887f6dffe160-exec-5   1/1     Running   0          7s


In [6]:
# Read in local file

dataset = pd.read_csv('/data/leggerf/Higgs100k.csv')
df = spark_session.createDataFrame(dataset)

%time total_events = df.count()

print('There are '+str(total_events)+' events')

CPU times: user 36.4 ms, sys: 15.5 ms, total: 51.9 ms
Wall time: 3.77 s
There are 99999 events


## Create parquet files, which are faster to read than CSV

Read [here](https://operational-intelligence.web.cern.ch/DataFormats) for a description of the most common data formats and their use

In [7]:
# read from HDFS
inputFile = 'hdfs://192.168.2.39/data/Higgs1M.csv'

%time df = spark_session.read.format('csv').option('header', 'true').option('inferschema', 'true').load(inputFile)

CPU times: user 48.3 ms, sys: 13.9 ms, total: 62.2 ms
Wall time: 9.82 s


In [8]:
#write parquet
hdfs_path=%hdfs_path
outputFile = hdfs_path+'/Higgs1M.parquet'

df.write.parquet(outputFile, mode='overwrite')

## Read in parquet files

how much faster is it to read parquet files rather than csv?

In [9]:
inputFile = 'hdfs://192.168.2.39/data/Higgs10M.parquet'
#inputFile = 'hdfs://192.168.2.39/data/Higgs1M.parquet'
#inputFile = 'hdfs://192.168.2.39/data/Higgs100k.parquet'
#other files are: Higgs1M.parquet, Higgs100k.parquet

%time df = spark_session.read.format('parquet').option('header', 'true').option('inferschema', 'true').load(inputFile)

CPU times: user 7.25 ms, sys: 3.07 ms, total: 10.3 ms
Wall time: 486 ms


In [10]:
#In how many partitions is the dataframe distributed?
print("Number of partitions: {}".format(df.rdd.getNumPartitions()))
print("Partitioner: {}".format(df.rdd.partitioner))

Number of partitions: 26
Partitioner: None


# Exercise 2

## Get familiar with spark workers

- Try to stop spark with `spark.stop()`, and start it again running the cell above with a different number of workers. What happens when you run `kubectl get pods`? 
- You can play with the number of workers, and run any of the cells that execute spark commands. Provided you're getting all the workers you're asking for, does the execution time change? Try to make some scaling tests

# Now let's have a look at the input data

In [11]:
%time total_events = df.count()

print('There are '+str(total_events)+' events')

CPU times: user 7.06 ms, sys: 6.56 ms, total: 13.6 ms
Wall time: 963 ms
There are 9560096 events


In [12]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- lepton_pT: double (nullable = true)
 |-- lepton_eta: double (nullable = true)
 |-- lepton_phi: double (nullable = true)
 |-- missing_energy_magnitude: double (nullable = true)
 |-- missing_energy_phi: double (nullable = true)
 |-- jet1_pt: double (nullable = true)
 |-- jet1_eta: double (nullable = true)
 |-- jet1_phi: double (nullable = true)
 |-- jet1_b-tag: double (nullable = true)
 |-- jet2_pt: double (nullable = true)
 |-- jet2_eta: double (nullable = true)
 |-- jet2_phi: double (nullable = true)
 |-- jet2_b-tag: double (nullable = true)
 |-- jet3_pt: double (nullable = true)
 |-- jet3_eta: double (nullable = true)
 |-- jet3_phi: double (nullable = true)
 |-- jet3_b-tag: double (nullable = true)
 |-- jet4_pt: double (nullable = true)
 |-- je4_eta: double (nullable = true)
 |-- jet4_phi: double (nullable = true)
 |-- jet4_b-tag: double (nullable = true)
 |-- m_jj: double (nullable = true)
 |-- m_jjj: double (nullable = true)
 |-- m_lv: 

# Exercise 3

- Create a function that makes a plot of any of the above variable for signal versus background (using the label variable to discriminate)
  - see an example of the plot in the hands-on [slides](https://github.com/leggerf/MLCourse-2122/blob/main/Slides/Day1/Big%20data%20science%20-%20Hands-on%20-%20Day%201%20-%202122.pdf)
  - the function should take as input the dataframe *df* and the variable name. For example `plotSignalvsBg(df, 'm_bb')`
  - to develop the code, use the 100k dataset, so that debugging goes quicker
- try to plot a few input variables and try to understand which ones are more promising to distinguish signal from background  

# Exercise 4 - Bonus

#### Create the input feature vector

- Libraries for ML tipically take as inputs data in a very specific format. Documentation on how to do data preprocessing in Spark: https://spark.apache.org/docs/latest/ml-features.html
- Try to add to the dataframe df a new column, called 'features' which is a vector column with all the variables above except for 'label'
   - features = [lepton_pT, lepton_eta, lepton_phi, ...]
   - Hint: look at the VectorAssembler transformer

# Exercise 5 - Bonus

Print (or draw) the correlation matrix (a table showing correlation coefficients between variables)

In [13]:
#when you're done, stop spark, this will release the resources you're using
spark.stop()